In [ ]:
# Step 1: Install Required Dependencies
%pip install langchain
%pip install langgraph
%pip install langchain-openai
%pip install chromadb
%pip install python-dotenv
%pip install pydantic

In [3]:
# Step 2: Import Core Libraries and Configure Environment
import os
import json
from datetime import datetime
from typing import List, Dict, Tuple, TypedDict, Annotated, Sequence
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import BaseMessage
from langchain.schema import Document
from langchain_community.vectorstores import Chroma

from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END

from pydantic import BaseModel, Field

# Load environment variables
load_dotenv(dotenv_path='env.txt')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Initialize models
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
embeddings = OpenAIEmbeddings()
output_parser = StrOutputParser()

In [5]:
# Step 3: 
# Agent State Definition
class AgentState(TypedDict):
    """Complete state container for multi-memory agent"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    working_memory: dict
    episodic_recall: list
    semantic_facts: dict
    user_id: str
    conversation_id: str

# Semantic Fact Structure
class SemanticFact(BaseModel):
    """Structure for semantic memory facts"""
    subject: str = Field(description="Entity or topic")
    predicate: str = Field(description="Relationship or property")
    object: str = Field(description="Value or related entity")
    confidence: float = Field(description="Confidence score 0-1")
    source: str = Field(description="Source: user or assistant")

# Initialize Vector Store
vector_store = Chroma(
    collection_name="agent_memory",
    embedding_function=embeddings,
    persist_directory="./memory_store"
)

In [6]:
# Step 4: 
def store_episodic_memory(vector_store, conversation_id: str, messages: List, summary: str = None):
    """Store conversation episode in vector memory"""
    if not summary and messages:
        summary = f"Conversation about: {messages[0][1] if isinstance(messages[0], tuple) else messages[0].content[:100]}..."
    
    metadata = {
        "type": "episodic",
        "conversation_id": conversation_id,
        "timestamp": datetime.now().isoformat(),
        "message_count": len(messages)
    }
    
    conversation_text = ""
    for msg in messages:
        if isinstance(msg, tuple):
            conversation_text += f"{msg[0]}: {msg[1]}\n"
        else:
            conversation_text += f"{msg.type}: {msg.content}\n"
    
    doc = Document(page_content=conversation_text, metadata=metadata)
    vector_store.add_documents([doc])
    return conversation_id

def retrieve_episodic_memories(vector_store, query: str, k: int = 3):
    """Retrieve relevant past conversation episodes"""
    results = vector_store.similarity_search(
        query=query,
        k=k,
        filter={"type": {"$eq": "episodic"}}
    )
    return results

In [7]:
# Step 5: Semantic Memory Functions
def extract_semantic_facts(messages: List) -> List[SemanticFact]:
    """Extract factual knowledge from conversation"""
    extraction_prompt = PromptTemplate.from_template("""
    Analyze this conversation and extract important factual statements.
    
    Conversation: {conversation}
    
    Extract facts in JSON format:
    {{"facts": [{{"subject": "...", "predicate": "...", "object": "...", 
                  "confidence": 0.0-1.0, "source": "user or assistant"}}]}}
    
    Only extract clear facts. Output valid JSON only.
    """)
    
    conversation_text = ""
    for msg in messages:
        if isinstance(msg, tuple):
            conversation_text += f"{msg[0]}: {msg[1]}\n"
        else:
            conversation_text += f"{msg.type}: {msg.content}\n"
    
    parser = JsonOutputParser()
    chain = extraction_prompt | llm | parser
    
    try:
        result = chain.invoke({"conversation": conversation_text})
        facts = [SemanticFact(**fact_dict) for fact_dict in result.get("facts", [])]
        return facts
    except Exception as e:
        print(f"Fact extraction error: {e}")
        return []

def store_semantic_facts(vector_store, facts: List[SemanticFact], user_id: str = "default"):
    """Store semantic facts in vector memory"""
    documents = []
    for fact in facts:
        fact_text = f"{fact.subject} {fact.predicate} {fact.object}"
        metadata = {
            "type": "semantic",
            "user_id": user_id,
            "subject": fact.subject,
            "predicate": fact.predicate,
            "object": fact.object,
            "confidence": fact.confidence,
            "timestamp": datetime.now().isoformat()
        }
        doc = Document(page_content=fact_text, metadata=metadata)
        documents.append(doc)
    
    if documents:
        vector_store.add_documents(documents)
    return len(documents)

def retrieve_semantic_facts(vector_store, query: str, user_id: str = "default", k: int = 5):
    """Retrieve relevant semantic facts"""
    results = vector_store.similarity_search(
        query=query, k=k,
        filter={"$and": [
            {"type": {"$eq": "semantic"}},
            {"user_id": {"$eq": user_id}}
        ]}
    )
    
    facts = []
    for doc in results:
        facts.append({
            "subject": doc.metadata.get("subject"),
            "predicate": doc.metadata.get("predicate"),
            "object": doc.metadata.get("object"),
            "confidence": doc.metadata.get("confidence", 1.0)
        })
    return facts

def format_semantic_context(facts: List[Dict]) -> str:
    """Format semantic facts for prompt inclusion"""
    if not facts:
        return "No relevant facts found."
    
    context = "Known facts:\n"
    for fact in facts:
        if fact.get('confidence', 1.0) > 0.7:
            context += f"- {fact['subject']} {fact['predicate']} {fact['object']}\n"
    return context

In [ ]:
# Step 6: Unified Memory Agent
def unified_memory_agent(state: AgentState) -> dict:
    """Agent with episodic and semantic memory capabilities"""
    
    current_messages = state.get("messages", [])
    user_id = state.get("user_id", "default")
    conversation_id = state.get("conversation_id", f"conv_{datetime.now().timestamp()}")
    
    # Retrieve context from both memory types
    episodic_context = ""
    semantic_context = ""
    
    if current_messages:
        # Get latest query
        if isinstance(current_messages[-1], tuple):
            latest_query = current_messages[-1][1]
        else:
            latest_query = current_messages[-1].content
        
        # Retrieve episodic memories
        past_episodes = retrieve_episodic_memories(vector_store, latest_query, k=2)
        if past_episodes:
            episodic_context = "Relevant past conversations:\n"
            for episode in past_episodes:
                timestamp = episode.metadata.get('timestamp', 'Unknown')
                episodic_context += f"[{timestamp}]:\n{episode.page_content[:200]}...\n\n"
        
        # Retrieve semantic facts
        facts = retrieve_semantic_facts(vector_store, latest_query, user_id=user_id, k=3)
        semantic_context = format_semantic_context(facts)
    
    # Create memory-augmented prompt
    memory_prompt = PromptTemplate.from_template("""
You are an AI assistant with both episodic and semantic memory.

{semantic_context}

{episodic_context}

Current conversation:
{messages}

Respond using your memories when relevant. Be consistent with known facts and past conversations.
""")
    
    # Format current messages
    formatted_messages = ""
    for msg in current_messages[-5:] if current_messages else []:
        if isinstance(msg, tuple):
            formatted_messages += f"{msg[0]}: {msg[1]}\n"
        else:
            formatted_messages += f"{msg.type}: {msg.content}\n"
    
    # Generate response
    chain = memory_prompt | llm | output_parser
    response = chain.invoke({
        "semantic_context": semantic_context,
        "episodic_context": episodic_context,
        "messages": formatted_messages
    })
    
    # Store conversation as episodic memory (after 2+ exchanges)
    if len(current_messages) >= 2:
        store_episodic_memory(vector_store, conversation_id, current_messages)
    
    # Extract and store new semantic facts
    if current_messages:
        messages_with_response = current_messages + [("assistant", response)]
        new_facts = extract_semantic_facts(messages_with_response[-3:])
        if new_facts:
            stored = store_semantic_facts(vector_store, new_facts, user_id)
            state["semantic_facts"] = {"extracted": stored}
    
    return {
        "messages": [("assistant", response)],
        "episodic_recall": past_episodes if past_episodes else [],
        "semantic_facts": state.get("semantic_facts", {})
    }

# Build workflow
memory_workflow = StateGraph(AgentState)
memory_workflow.add_node("memory_agent", unified_memory_agent)
memory_workflow.set_entry_point("memory_agent")
memory_workflow.add_edge("memory_agent", END)

# Compile application
memory_app = memory_workflow.compile()

In [9]:
# Test conversation 1: Establish facts
test_1 = {
    "messages": [
        ("user", "Hi, I'm Sarah Chen. I'm a data scientist working on climate models. I'm vegetarian and prefer concise technical explanations."),
    ],
    "user_id": "sarah_chen",
    "conversation_id": "conv_001"
}

result_1 = memory_app.invoke(test_1)
print("Response 1:")
print(result_1["messages"][-1][1] if isinstance(result_1["messages"][-1], tuple) else result_1["messages"][-1].content)

print("\n" + "="*50 + "\n")

# Test conversation 2: Use semantic memory
test_2 = {
    "messages": [
        ("user", "What machine learning techniques would you recommend for time series forecasting?"),
    ],
    "user_id": "sarah_chen",
    "conversation_id": "conv_002"
}

result_2 = memory_app.invoke(test_2)
print("Response 2 (with semantic memory):")
print(result_2["messages"][-1][1] if isinstance(result_2["messages"][-1], tuple) else result_2["messages"][-1].content)

print("\n" + "="*50 + "\n")

# Test conversation 3: Reference past conversation
test_3 = {
    "messages": [
        ("user", "Can you recommend some lunch options for our team meeting?"),
    ],
    "user_id": "sarah_chen",
    "conversation_id": "conv_003"
}

result_3 = memory_app.invoke(test_3)
print("Response 3 (with episodic and semantic memory):")
print(result_3["messages"][-1][1] if isinstance(result_3["messages"][-1], tuple) else result_3["messages"][-1].content)

Response 1:
Hi Sarah! Great to meet you. As a data scientist working on climate models, I can provide concise technical explanations tailored to your work. If you have any specific questions or topics you'd like to discuss—whether about climate modeling techniques, data processing, or anything else—just let me know. Also, if you want any vegetarian-friendly recommendations or tips related to your preferences, feel free to ask!


Response 2 (with semantic memory):
For time series forecasting, I recommend these machine learning techniques:

1. **ARIMA/SARIMA** – Traditional statistical models effective for linear patterns and seasonality.
2. **LSTM (Long Short-Term Memory networks)** – Deep learning models that capture long-term dependencies in sequential data.
3. **Prophet** – A decomposable time series model by Facebook, good for handling seasonality and holidays with minimal tuning.
4. **XGBoost/LightGBM** – Gradient boosting methods that work well when you engineer time-based feature